In [1]:
import gym
import collections
import random
import time
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors
from IPython.display import clear_output
import time
import csv
from gym import envs
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import BatchNormalization, Reshape, UpSampling2D, Conv2DTranspose, LeakyReLU, ZeroPadding2D, Input
from keras.layers.convolutional import Conv2D
from keras.layers.recurrent import LSTM, GRU
from keras.models import Sequential, load_model, Model
from keras import regularizers, optimizers
from keras.optimizers import RMSprop, Adam
from keras.models import Sequential, load_model
from keras import regularizers, optimizers
from keras.utils import to_categorical
np.random.seed(1234)
import scipy.io as sio
import scipy as scp
from scipy import stats
import operator
from keras import backend as K
import tensorflow as tf
import os
import keras
from keras.models import load_model
from IPython.display import display, clear_output
from keras.backend.tensorflow_backend import set_session
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0'
#config=tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction=0.5
#set_session(tf.Session(config=config))

Using TensorFlow backend.


In [ ]:
all_envs = envs.registry.all()
env_ids = [env_spec.id for env_spec in all_envs]
print(np.transpose(env_ids)) 

In [2]:
# FF
class DQNAgent:
    def __init__(self,a):
        #(210,160,3),6
        self.drop = 0
        self.state_size = (128,)
        self.action_size = 4 # 9 for pacman, 6 for space invader
        self.memory = collections.deque(maxlen=1000000)
        self.gamma = 0.99    # discount rate 0.95
        self.epsilon = 1  # exploration rate 1 
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.9998
        self.learning_rate = 10**-3
        self.time_penalty = 0
        self.model = self._build_model()
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        
        model.add(Dense(512, input_shape=self.state_size, activation='relu'))
        #model.add(Dropout(self.drop))
        
        #model.add(Flatten())
        
        model.add(Dense(128, activation='relu'))
        #model.add(Dropout(self.drop))
        
        #model.add(Dense(32, activation='relu'))
        #model.add(Dropout(self.drop))
        
        model.add(Dense(self.action_size, activation='linear'))
        
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        model.summary()
        return model
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) #previous state, action, reward after action
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            act_values = np.zeros((1,self.action_size))
            return act_values,env.action_space.sample()
        state = np.expand_dims(state,axis=0)
        #print(state.shape)
        act_values = self.model.predict(state)
        #return np.argmax(act_values[0])  # returns action
        return act_values,np.argmax(act_values)   # returns action
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            state = np.expand_dims(state,axis=0)
            next_state = np.expand_dims(next_state,axis=0)
            target = reward #if we are done the final reward is our target
            if not done: #if we are not done the target is the current reward plut the predicted reward from the next state
                target = reward + self.gamma * np.amax(self.model.predict(next_state)) #two frames ahead
                #print(self.gamma * np.amax(self.model.predict(next_state)))
            target_f = self.model.predict(state) 
            #print(target_f)
            target_f[0][action] = target- self.time_penalty
            #print(target_f)
            print(state.shape)
            print(target_f.shape)
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [4]:
#env = gym.make('SpaceInvaders-v0')
env = gym.make('Breakout-ramDeterministic-v0')


In [5]:
balls = np.zeros(1000)
for i in  range(1000):
    balls[i] = env.action_space.sample()
np.max(balls)+1

4.0

In [6]:
state = env.reset()
next_state, reward, done, info = env.step(1)
lives_before = info['ale.lives']
print(info)
#env.render()
#plt.imshow(state)
state.shape

{'ale.lives': 5}


(128,)

In [7]:
agent = DQNAgent(env)
episodes = 10000
death_penalty = 40
agent.epsilon_decay = 0.9996


# initialize gym environment and the agent
env = gym.make('Breakout-ramDeterministic-v0')
#env = gym.make('SpaceInvaders-v0')
total_reward_counter = np.empty([episodes])
Q_reward_counter = np.empty([episodes])
reward = 0

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               66048     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 516       
Total params: 132,228
Trainable params: 132,228
Non-trainable params: 0
_________________________________________________________________


In [8]:
#agent.epsilon = 1
# Iterate the game
for e in range(episodes):
    total_reward = 0
    
    # reset state in the beginning of each game
    state = env.reset()
    state, reward, done, info = env.step(0)
    #state = scp.stats.zscore(state)
    lives_before = info['ale.lives']
    #state = state[:,:,0]
    

    #state = np.reshape(state, [1, 4])
    # time_t represents each frame of the game
    # Our goal is to keep the pole upright as long as possible until score of 500
    # the more time_t the more score
    for time_t in range(1000):
        reward_cum = 0
        # turn this on if you want to render
        #env.render()
        #time.sleep(0.01)
        # Decide action
        #state = np.expand_dims(state,axis=0)
        
        actvals, action = agent.act(state)
        #print(actvals)
        #print(action)
        #action = np.argmax(actvals)
   
        # Advance the game to the next frame based on the action.
        # Reward is 1 for every frame the pole survived
        
        next_state, reward, done, info = env.step(action)
        #next_state = scp.stats.zscore(next_state)
        #if reward>0:
        #    reward = 1
        reward_cum += reward
        
        #next_state = np.expand_dims(next_state,axis=1)
        

        #next_state = next_state[:,:,0]
        
        #print(next_state.shape)

        
        
        lives_after = info['ale.lives']
        
        #if lives_after < lives_before:
        #    reward_cum = reward_cum -death_penalty
            
        total_reward = total_reward + reward_cum    
        #next_state = np.reshape(next_state, [1, 4])
        # Remember the previous state, action, reward, and done
        agent.remember(state, action, reward_cum, next_state, done)
        lives_before = lives_after
        # make next_state the new current state for the next frame.
        state = next_state
        # done becomes True when the game ends
        # ex) The agent drops the pole
        if done:
            # print the score and break out of the loop
            #clear_output(wait=True)
            print("episode: {}/{}, score: {}, epsilon: {}".format(e+1, episodes, total_reward, agent.epsilon))
            #print(state.shape)
            #print(state.shape)
            #plt.plot(state)
            #state = np.expand_dims(state,axis=0)
            #state = np.expand_dims(state,axis=2)
            #print(state.shape)
            #plt.plot(state)
            #state = np.expand_dims(state,axis=4)
            Q_reward_counter[e] = np.max(actvals)
            total_reward_counter[e] = total_reward

            break
    # train the agent with the experience of the episode
    agent.replay(32)

episode: 1/10000, score: 1.0, epsilon: 1
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
episode: 2/10000, score: 0.0, epsilon: 0.9996
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(1, 128)
(1, 4)
(

KeyboardInterrupt: 

In [ ]:
#print(np.max(Q_reward_counter))
plt.plot(Q_reward_counter)
plt.axis([0, Q_reward_counter.shape[0], np.min(Q_reward_counter), np.max(Q_reward_counter)])


In [ ]:
window = 50
rew_avg = np.empty([episodes])
for pnts in range(window,total_reward_counter.shape[0]):
    rew_avg[pnts] =  np.mean(total_reward_counter[(pnts-window) : pnts+window])
plt.plot(total_reward_counter)
plt.plot(rew_avg)
plt.axis([0, total_reward_counter.shape[0], np.min(rew_avg), np.max(rew_avg)])
np.argmax(rew_avg)

In [ ]:
#playtest
frms = 0
env = gym.make('Breakout-ramDeterministic-v0')

#agent = DQNAgent(env)
#agent.model = keras.models.load_model('breakoutfeedforwardRAM1k.h5')

#envisual = gym.make('BreakoutDeterministic-v0')
state = env.reset()
#statevisual = envisual.reset()



plt.ion()
fig = plt.figure()


plt.axis([0, 432, 0, 288])
agent.epsilon = 0.05;
actgrid = [[0,0,0],[0,0,0]]
#for e in range(10000):
#    state = env.reset()
#statevisual = envisual.reset()
for time_t in range(10000):
    clear_output(wait=True)

    #state = np.expand_dims(state,axis=4)


    actvals, action = agent.act(state)
    state = np.expand_dims(state,axis=0)
    expectation = np.amax(agent.model.predict(state))
    #print(state)

    #print(agent.model.predict(np.random.rand(1,128)))

    #actgrid[0][0] = actvals[0][5]
    #actgrid[0][1] = actvals[0][1]
    #actgrid[0][2] = actvals[0][4]

    actgrid[0][0] = actvals[0][3]
    actgrid[0][1] = actvals[0][0]
    actgrid[0][2] = actvals[0][2]

    actgrid[1][0] = actvals[0][3]
    actgrid[1][1] = actvals[0][0]
    actgrid[1][2] = actvals[0][2]

    #actgrid[2][0] = actvals[0][8]
    #actgrid[2][1] = actvals[0][4]
    #actgrid[2][2] = actvals[0][7]

    #action = 3
    state, reward1, done, _ = env.step(action)
    #statevisual, reward2, done, info = envisual.step(action)
    #state, reward, done, _ = env.step(0)

    #print(actvals)
    #print(actgrid)

    #f, axarr = plt.subplots(1,2)

    #cmap = colors.Colormap('magma',N = 256)
    #norm = colors.Normalize(np.min(actgrid),np.max(actgrid),clip=True)
    #print(actgrid)
    #print(zactgrid)
    #axarr[0].imshow(actgrid,vmin = np.min(actgrid), vmax = np.max(actgrid))
    plt.imshow(actgrid,vmin = np.min(actgrid), vmax = np.max(actgrid))
    #axarr[1].imshow(statevisual)
    env.render()



    plt.show()
    print(action)
    print(expectation)
    #print(info)
    print(done)
    #print(reward1)
    #print(reward2)
    #time.sleep(0.001)
    #print(action)


    frms = frms+1

    if done:
        env.close()
        break
        

In [ ]:
#agent.model.save('breakoutfeedforwardRAM10k.h5')